# OpenAI Assistant API

## Install OpenAI API

In [ ]:
!pip install -q openai  > /dev/null

In [ ]:
from openai import OpenAI
from google.colab import userdata

# Create OpenAI client
client = OpenAI(api_key=userdata.get('openaiKey'))

## Reading Data

Note that REST API of OpenAI returns paged results. Therefore, we have to loop over the pages.


In [ ]:
# For demo purposes, we limit the page size to 3 items.
assistants = client.beta.assistants.list(limit=3, order="asc")

while True:
    if assistants.first_id:
        # For illustration purposes, we print the first and list ID of the page
        print(f"Page from {assistants.first_id} to {assistants.last_id}")

    # Print all records of the page
    for a in assistants.data:
        print(f"\t{a.name} ({a.id})")

    if not assistants.has_next_page():
        # No more pages, exit the loop
        break

    # Get next page
    print("Reading next page...")
    assistants = assistants.get_next_page()


## Helper Methods

In [ ]:
def get_assistant(name, client):
    """Checks if OpenAI assistant with given name exists.

    Returns assistant or None if it does not exist.
    """
    assistants = client.beta.assistants.list()
    while True:
        for assistant in assistants.data:
            if assistant.name == name:
                return assistant

        if not assistants.has_next_page():
            break

        assistants = assistants.get_next_page()

    return None

def get_file(name, client):
    """Checks if file with given name exists.

    Returns file or None if it does not exist.
    """
    files = client.files.list()
    file = [ x for x in files if x.filename == name]
    if file:
        return file[0]

    return None

## Handling Files

Files can be added to assistants and messages. They are useful for retrieval and
code interpreter tools.

In [ ]:
file = get_file("qualifikationsprofil.md", client)
if not file:
    print("File does not exist yet, uploading it...")
    file = client.files.create(file=open("qualifikationsprofil.md", "rb"), purpose="assistants")
    print(f"New file uploaded ({file.id})")
else:
    print(f"File already exists ({file.id})")

## Create/Update Assistant

In [ ]:
ASSISTANT_NAME = "Bildungsberatungsassistent"
ASSISTANT_DESC = "Assistent, der SchülerInnen bei der Schulauswahl hilft"
ASSISTANT_MODEL = "gpt-4-1106-preview"
ASSISTANT_INSTRUCTIONS = """
Du bist ein Assistent, der SchülerInnen hilft, festzustellen, ob unsere Schule
die richtige für sie ist. Dafür beantworte Fragen zum Qualifikationsprofil
unserer AbsolventInnen auf Basis der Datei _qualifikationsprofil.md_. Beantworte
NUR Fragen zum Qualifikationsprofil. Verwende NUR die Informationen aus der
Datei.
"""
METADATA = {"project": "OpenAI API Video Tutorial"}

assistant_data = {
    "model": ASSISTANT_MODEL,
    "name": ASSISTANT_NAME,
    "description": ASSISTANT_DESC,
    "instructions": ASSISTANT_INSTRUCTIONS,
    "metadata": METADATA,
    "tools": [{"type": "retrieval"}],
    "file_ids": [file.id],
}

assistant = get_assistant(ASSISTANT_NAME, client)
if not assistant:
    print("Assistant does not exist yet, creating it...")
    assistant = client.beta.assistants.create(**assistant_data)
    print(f"New assistant created ({assistant.id})")
else:
    print(f"Assistant already exists ({assistant.id}), updating it...")
    assistant_data["assistant_id"] = assistant.id
    assistant = client.beta.assistants.update(**assistant_data)
    print(f"Assistant updated")


## Answer Question With Retrieval Tool

In [ ]:
import time
import pprint

QUESTION = "Haben AbsolventInnen der Schule eine Ausbildung bzgl. Projektmanagement?"

# Here we create the thread and the run separately. Note that there is an
# API (https://platform.openai.com/docs/api-reference/runs/createThreadAndRun)
# with which you could create a thread and start a run in one go.

thread = client.beta.threads.create(
    messages=[{"role": "user", "content": QUESTION}],
    metadata=METADATA,
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    metadata=METADATA,
)

while True:
    print("Sleeping for a moment to give OpenAI time to think...")
    time.sleep(3)

    run = client.beta.threads.runs.retrieve(
        run_id=run.id,
        thread_id=thread.id,
    )
    print(f"Run status: {run.status}")

    # Note that the thread API returns a paged result. To keep things simple,
    # we assume that no paging is necessary (i.e. we have less than 100
    # steps).
    steps = client.beta.threads.runs.steps.list(
        run_id=run.id,
        thread_id=thread.id,
        limit=100
    )
    for step in steps:
        print(f"\t{step.type}: {step.status}")

    if run.status == "completed":
        break;

# Get the last message and print it on the screen
messages = client.beta.threads.messages.list(
    thread_id=thread.id,
    limit=100)
pprint.pprint(messages.data[0].content[0].text.value)

# Note that you can get annotations, too. In the case of the retrieval tool,
# annotations contain sources from uploaded files.
messages.data[0].content[0].text.annotations[0]